In [2]:
import wikipedia # libreria de python de wikipedia
from datetime import date # para saber el anio actual
import re #se usa para poder reemplazar los simbolos por espacios en un string
import requests
import json


In [3]:
# CONSTANTES

wikipedia.set_lang("es") # setea el lenguaje en que se va a buscar en wikipedia

todays_date = date.today()

meses_nombres = {'enero': 1,'febrero': 2,'marzo': 3,'abril': 4,'mayo': 5,'junio': 6,'julio': 7,'agosto': 8,'septiembre': 9,'octubre': 10,'noviembre': 11,'diciembre': 12}
key_list = list(meses_nombres.keys())

signo = ("Capricornio", "Acuario", "Piscis", "Aries", "Tauro", "Géminis", "Cáncer", "Leo", "Virgo", "Libra", "Escorpio", "Sagitario")
fechas = (20, 19, 20, 20, 21, 21, 22, 22, 22, 22, 22, 21)

WIKI_REQUEST_IMAGEN = 'http://en.wikipedia.org/w/api.php?action=query&prop=pageimages&format=json&piprop=original&titles='
WIKI_REQUEST_INFOBOX = 'https://en.wikipedia.org/w/api.php?action=query&prop=revisions&rvprop=content&format=json&titles='



In [4]:
# funcion que devuelve el numero del mes que se ingresa y el nombre

def devolver_nombre_mes(num):
    return key_list[num-1] # busca el mes dentro del diccionario y se guarda el nombre


In [5]:
# devuelve el signo del zodiaco segun dia y mes

def calcular_signo(mes_numero,dia):
    mes_numero=mes_numero-1
    if dia>fechas[mes_numero]:
        mes_numero=mes_numero+1
    if mes_numero==12:
        mes_numero=0
    signo_persona = signo[mes_numero] # guarda en una variable el signo de la persona
    return signo_persona

In [6]:
#calcula la edad de la persona

def calcular_edad(anio):
    edad = todays_date.year - anio #resta anio actual menos anio de nacimiento
    return edad

In [7]:
# devuelve todas las coincidencias

persona = input('Ingrese persona: ')

lista_de_coincidencias = wikipedia.search(persona)  #busca todas las coincidencias que exiten en wikipedia con el nombre

for i in range(0,len(lista_de_coincidencias)): # imprime todas las coincidencias de wikipedia con el nombre ingresado
    print('Opcion',i+1,":",lista_de_coincidencias[i])

Ingrese persona: calcuta
Opcion 1 : Calcuta
Opcion 2 : Teresa de Calcuta
Opcion 3 : Mohammedan SC Calcuta
Opcion 4 : Universidad de Calcuta
Opcion 5 : Jardín botánico de Calcuta
Opcion 6 : Copa Calcuta
Opcion 7 : Iglesia de Santo Tomás (Calcuta)
Opcion 8 : Servicio de autobús Londres–Calcuta
Opcion 9 : Puerto de Calcuta
Opcion 10 : Agujero negro de Calcuta


In [8]:
# se queda con la pagina de la persona ingresada

opcion = int(input('Ingrese Opcion: '))
pagina = wikipedia.WikipediaPage(lista_de_coincidencias[opcion-1]) # se queda con la pagina de wikipedia de la persona

Ingrese Opcion: 2


In [10]:
## EMPIEZA A CALCULAR LAS FECHAS

response  = requests.get(WIKI_REQUEST_INFOBOX + pagina.title)
json_data = json.loads(response.text)
json_infobox = list(json_data['query']['pages'].values())[0]['revisions'][0]['*']
info_box = json_infobox.split("Infobox")[1] # se queda con la info de la seccion "infobox" de wikipedia

KeyError: 'revisions'

In [ ]:
## ---------- NACIMIENTO ----------- ##
birth_date = json_infobox.split("birth_date")[1]
birth_date = birth_date.split("}")[0] # se queda con la fecha de cumpleaños de la persona

birth_date = re.sub('[^a-zA-Z0-9\n\.]', ' ', birth_date) # cambia todos los simbolos por espacios

numeros_fechas_cumple = [int(i) for i in birth_date.split() if i.isdigit()] # se queda con todos los numeros del string

anio_nacimiento = numeros_fechas_cumple[0]
mes_nacimiento = numeros_fechas_cumple[1]
dia_nacimiento = numeros_fechas_cumple[2]

print(dia_nacimiento,mes_nacimiento,anio_nacimiento)

In [ ]:
## ----------- MUERTE -------------- ##
try: 
    birth_date = json_infobox.split("death_date")[1]
    birth_date = birth_date.split("}")[0]
    birth_date=re.sub('[^a-zA-Z0-9\n\.]', ' ', birth_date) # cambia todos los simbolos por espacios
    numeros_fechas_muerte = [int(i) for i in birth_date.split() if i.isdigit()] # se queda con todos los numeros del string

    anio_muerte = numeros_fechas_muerte[0]
    mes_muerte = numeros_fechas_muerte[1]
    dia_muerte = numeros_fechas_muerte[2]

    print(dia_muerte,mes_muerte,anio_muerte)
    
except:
    print('La persona no murió.')
    anio_muerte = 0
    

In [ ]:
## ---------- IMAGEN --------------- ##

response  = requests.get(WIKI_REQUEST_IMAGEN + pagina.title)
json_data = json.loads(response.text)
imagen_persona = list(json_data['query']['pages'].values())[0]['original']['source']

print(imagen_persona)

In [ ]:
# dice toda la informacion de la persona

mes_nacimiento_nombre = devolver_nombre_mes(mes_nacimiento)
edad = calcular_edad(anio_nacimiento)
signo = calcular_signo(mes_nacimiento,dia_nacimiento)

print('---------- FICHA DE LA PERSONA ----------')

print('Nombre:',pagina.title) #titulo de la pagina de wikipedia

print('Fecha de Nacimiento:',dia_nacimiento,"de",mes_nacimiento_nombre,'de',anio_nacimiento)

if (anio_muerte != 0):
      mes_muerte_nombre = devolver_nombre_mes(mes_muerte)
      print('Fecha de Muerte:',dia_muerte,"de",mes_muerte_nombre,'de',anio_muerte)
      print('La persona tendría:',edad,'años.')
else:
    print('Edad:',edad)
    
print('Signo del Zodíaco:',signo)

print('Información extraída de:',pagina.url)

print('Imagen de la persona:',imagen_persona)

In [ ]:
## -------------------------------------------- PROGRAMA CON LAS DOS OPCIONES ------------------------------------------ ##

In [11]:
def opcion_2_sin_api():
    
        # se queda con la pagina de la persona ingresada y se empieza a desarmar la fecha de nacimiento
        
        resumen = pagina.summary # obtiene el resumen
        resumen_primera_parte = resumen.split('(', 1) # se queda con lo que esta en el primer parentesis (lo de la derecha)
        resumen_segunda_parte = resumen_primera_parte[1].split('.',1) # se queda con lo que esta hasta el primer punto (lo del izquierda)
        fecha = resumen_segunda_parte[0]
        #print(fecha)
        
        # desarma la fecha y se queda con el numero del dia
        dia = int([int(i) for i in fecha.split() if i.isdigit()][0]) # se queda con el primer numero que esta en el string
        #print(dia)
        
        # se queda con el anio de nacimiento

        fecha = re.sub('[^a-zA-Z0-9\n\.]', ' ', fecha) # cambia todos los simbolos por espacios
        numeros_fechas = [int(i) for i in fecha.split() if i.isdigit()] # se queda con todos los numeros del string

        #print(numeros_fechas)

        for i in range (0,len(numeros_fechas)):
            if int(numeros_fechas[i]) > 1000: # cuando encuentra el primer numero mayor a mil, corta
                anio = int(numeros_fechas[i]) # se guarda el anio de nacimiento
                break
        #print(anio)
        
        # devuelve el numero del mes que se ingresa y el nombre

        meses_nombres = {'enero': 1,'febrero': 2,'marzo': 3,'abril': 4,'mayo': 5,'junio': 6,'julio': 7,'agosto': 8,'septiembre': 9,'octubre': 10,'noviembre': 11,'diciembre': 12}

        lista = fecha.split(" ") # guarda en una lista todos los elementos del string

        for i in lista:
          if i in meses_nombres: # se fija cual de esos está dentro del diccionario de meses
            mes = i # se guarda el nombre del mes y corta
            break

        #print(mes)

        mes_numero = int(meses_nombres[mes.lower()]) # busca el mes dentro del diccionario y se guarda el numero
        #print(mes_numero)
        
        # devuelve el signo del zodiaco segun dia y mes

        signo_persona = calcular_signo(mes_numero,dia)
        
        #print ("Signo: " + signo_persona)
        
        # calcula la edad de la persona

        edad = calcular_edad(anio)
        #print(edad)
        
        # dice toda la informacion de la persona
        
        print('---------- FICHA DE LA PERSONA ----------')

        print('Nombre:',pagina.title) #titulo de la pagina de wikipedia
        
        print('Fecha de Nacimiento:',dia,"de",mes,'de',anio)
        
        print('Edad:',edad)
        
        print('Signo del Zodíaco:',signo_persona)

        print('Información extraída de:',pagina.url)
        

In [12]:
def opcion_1_usa_api():
        ## EMPIEZA A CALCULAR LAS FECHAS

        response  = requests.get(WIKI_REQUEST_INFOBOX + pagina.title)
        json_data = json.loads(response.text)
        json_infobox = list(json_data['query']['pages'].values())[0]['revisions'][0]['*']
        info_box = json_infobox.split("Infobox")[1] # se queda con la info de la seccion "infobox" de wikipedia
        
        ## ---------- NACIMIENTO ----------- ##
        birth_date = json_infobox.split("birth_date")[1]
        birth_date = birth_date.split("}")[0] # se queda con la fecha de cumpleaños de la persona

        birth_date = re.sub('[^a-zA-Z0-9\n\.]', ' ', birth_date) # cambia todos los simbolos por espacios

        numeros_fechas_cumple = [int(i) for i in birth_date.split() if i.isdigit()] # se queda con todos los numeros del string

        anio_nacimiento = numeros_fechas_cumple[0]
        mes_nacimiento = numeros_fechas_cumple[1]
        dia_nacimiento = numeros_fechas_cumple[2]

        #print(dia_nacimiento,mes_nacimiento,anio_nacimiento)
        
        ## ----------- MUERTE -------------- ##
        murio = False
        try: 
            birth_date = json_infobox.split("death_date")[1]
            birth_date = birth_date.split("}")[0]
            #print(birth_date)
            birth_date=re.sub('[^a-zA-Z0-9\n\.]', ' ', birth_date) # cambia todos los simbolos por espacios
            numeros_fechas_muerte = [int(i) for i in birth_date.split() if i.isdigit()] # se queda con todos los numeros del string

            anio_muerte = numeros_fechas_muerte[0]
            mes_muerte = numeros_fechas_muerte[1]
            dia_muerte = numeros_fechas_muerte[2]

            murio = True
            #print(dia_muerte,mes_muerte,anio_muerte)

        except:
            #print('La persona no murió.')
            #anio_muerte = 0
            pass
            
        ## ---------- IMAGEN --------------- ##
        
        response  = requests.get(WIKI_REQUEST_IMAGEN + pagina.title)
        json_data = json.loads(response.text)
        imagen_persona = list(json_data['query']['pages'].values())[0]['original']['source']

        #print(imagen_persona)
        
        # dice toda la informacion de la persona

        mes_nacimiento_nombre = devolver_nombre_mes(mes_nacimiento)
        #print(mes_nacimiento_nombre)
        edad = calcular_edad(anio_nacimiento)
        #print(edad)
        signo = calcular_signo(mes_nacimiento,dia_nacimiento)
        #print(signo)
        
        #print(murio)
        if (murio):
            mes_muerte_nombre = devolver_nombre_mes(mes_muerte)

        print('---------- FICHA DE LA PERSONA ----------')

        print('Nombre:',pagina.title) #titulo de la pagina de wikipedia

        print('Fecha de Nacimiento:',dia_nacimiento,"de",mes_nacimiento_nombre,'de',anio_nacimiento)

        #print(murio)
        if (murio):
            print('Fecha de Muerte:',dia_muerte,"de",mes_muerte_nombre,'de',anio_muerte)
            print('La persona tendría:',edad,'años.')
        else:
            print('Edad:',edad)

        print('Signo del Zodíaco:',signo)

        print('Información extraída de:',pagina.url)

        print('Imagen de la persona:',imagen_persona)

In [13]:
opcion_1 = True
try:
    #print('entro 1')
    opcion_1_usa_api()
except:
    #print('error en 1')
    opcion_1 = False
#print(opcion_1)    
if(opcion_1 == False):
    try:
        #print('entro 2')
        opcion_2_sin_api()
    except:
        print('No es posible encontrar la persona')

---------- FICHA DE LA PERSONA ----------
Nombre: Teresa de Calcuta
Fecha de Nacimiento: 26 de agosto de 1910
Edad: 111
Signo del Zodíaco: Virgo
Información extraída de: https://es.wikipedia.org/wiki/Teresa_de_Calcuta
